In [ ]:
from soview import *
import pandas as pd

In [ ]:
raw_path = 'zhiyuanyuan/PUBDT/st/merfish/cortex_science/doi_10'
h5ad_path = 'zhiyuanyuan/PUBDT/st/merfish/cortex_science/h5ad'

In [ ]:
sample_name = 'H18.06.006.MTG.250.expand.rep1'

In [ ]:
features_file = f'{raw_path}/{sample_name}.features.csv'
genes_file = f'{raw_path}/{sample_name}.genes.csv'
matrix_file = f'{raw_path}/{sample_name}.matrix.csv'

In [ ]:
features = pd.read_csv(features_file)

In [ ]:
genes = pd.read_csv(genes_file).values[:,0]

In [ ]:
matrix = pd.read_csv(matrix_file)

In [ ]:
mat = np.zeros(shape=(len(features),len(genes)))
for i in range(matrix.shape[0]):
    gene_idx = matrix.iloc[i]['row']
    cell_idx = matrix.iloc[i]['col']
    val = matrix.iloc[i]['val']
    mat[cell_idx-1][gene_idx-1]+=val


In [ ]:
np.array(features['name'])

In [ ]:
features

In [ ]:
adata = ad.AnnData(mat)
adata.obs_names = np.array(features['name'])
adata.var_names = genes
for col in features.columns:
    adata.obs[col] = np.array(features[col])
adata.obsm['spatial'] = features[['global.x','global.y']].values

In [ ]:
sc.pl.embedding(adata,basis='spatial',color=['cluster_L1','cluster_L2','cluster_L3'])

In [ ]:
sample_name_list = ['H18.06.006.MTG.250.expand.rep1','H18.06.006.MTG.4000.expand.rep1','H18.06.006.MTG.4000.expand.rep2',
                   'H18.06.006.MTG.4000.expand.rep3','H19.30.001.STG.250.expand.rep1','H19.30.001.STG.4000.expand.rep1',
                    'H19.30.001.STG.4000.expand.rep2','H20.30.001.STG.250.unexpand.rep1','H20.30.001.STG.250.unexpand.rep2',
                    'H20.30.001.STG.4000.expand.rep1','H20.30.001.STG.4000.expand.rep2','H20.30.001.STG.4000.expand.rep3',
                    'H22.26.401.MTG.250.unexpand.rep1','H22.26.401.MTG.4000.expand.rep1','H22.26.401.MTG.4000.expand.rep2',
                    'mouse1.AUD_TEA_VIS.242.unexpand','mouse2.AUD_TEA_VIS.242.unexpand'
                   ]
for sample_name in sample_name_list:
    features_file = f'{raw_path}/{sample_name}.features.csv'
    genes_file = f'{raw_path}/{sample_name}.genes.csv'
    matrix_file = f'{raw_path}/{sample_name}.matrix.csv'

    features = pd.read_csv(features_file)
    genes = pd.read_csv(genes_file).values[:,0]
    matrix = pd.read_csv(matrix_file)

    mat = np.zeros(shape=(len(features),len(genes)))
    for i in range(matrix.shape[0]):
        gene_idx = matrix.iloc[i]['row']
        cell_idx = matrix.iloc[i]['col']
        val = matrix.iloc[i]['val']
        mat[cell_idx-1][gene_idx-1]+=val

    adata = ad.AnnData(mat)
    adata.obs_names = np.array(features['name'])
    adata.var_names = genes
    for col in features.columns:
        adata.obs[col] = np.array(features[col])
    adata.obsm['spatial'] = features[['global.x','global.y']].values
    adata.write_h5ad(f'{h5ad_path}/{sample_name}.h5ad')
    sc.pl.embedding(adata,basis='spatial',color=['cluster_L1','cluster_L2','cluster_L3'])

In [ ]:
from soview import *
import shutil
import os
import gc

root_path = 'zhiyuanyuan/projects/SODB/'
source_h5ad_path = h5ad_path
biotech_category = 'Spatial_Transcriptomics'
biotech = 'MERFISH'
# 多个单词要加下划线
dataset_short = 'Fang2022Conservation'

h5ad_list = os.listdir(source_h5ad_path)
out_dataname_list = []
out_h5ad_path_list = []
out_feature_dict_list = []
N_Unit_list = []
N_Gene_list = []
for hl in h5ad_list:
    if not hl.endswith('.h5ad'):
        continue
    # source_h5ad_data = hl.split('.')[0]
    source_h5ad_data = hl[0:-5]


    source_h5ad = f'{source_h5ad_path}/{source_h5ad_data}.h5ad'

    data_name = source_h5ad_data
    
    import os

    path_rel = f'demo_data/{biotech_category}/{biotech}/{dataset_short}/{data_name}'
    path_abs = root_path+path_rel
    os.makedirs(path_abs, exist_ok=False)
    print("The new directory is created!")
    shutil.copyfile(source_h5ad, f'{path_abs}/data.h5ad')
    
    
    data_use = path_rel
    adata = ad.read_h5ad(f'{root_path}{data_use}/data.h5ad')
    print(adata.X)
    print(f'h5ad path: {data_use}/data.h5ad')
    
    adata.obsm['spatial'] = np.array(adata.obsm['spatial'][:,[0,1]])
    
    adata.layers["counts"] = adata.X.copy()
    
    # if np.sum(adata.X)==np.sum(adata.X.astype('int')):
    if True:
        # is all the element in adata.X are integer?
    
        sc.pp.normalize_per_cell(adata)
        sc.pp.log1p(adata)
    # if adata.shape[1]>=2000:
    #     sc.pp.highly_variable_genes(adata, flavor="seurat", n_top_genes=2000)
    # else:
    #     pass
    

    sc.pp.pca(adata)
    sc.pp.neighbors(adata)
    sc.tl.leiden(adata)
    
    sc.pl.embedding(adata,basis='spatial',color='leiden')
    sc.tl.umap(adata,n_components=3)
    SOViewer_plot(adata,embedding_use='X_umap',dot_size=10)
    print(f'View path: {data_use}/View_UMAP_LAB.pdf')
    
    gene_list = np.array(adata.var_names).tolist()
    obs_list = np.array(adata.obs.keys()).tolist()
    obsm_list = []
    for obsm in np.array(adata.obsm.keys()).tolist():
        for i in range(adata.obsm[obsm].shape[1]):
            cur_obsm = f'{obsm}@{i}'
            obsm_list.append(cur_obsm)
            
    feature_dict = {
        'gene':gene_list,
        'obs':obs_list,
        'obsm':obsm_list
    }
    
    import pickle
    with open(f'{root_path}{data_use}/feature_dict.pickle', 'wb') as handle:
        pickle.dump(feature_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)

    print(f'把feature_dict写为: {data_use}/feature_dict.pickle')
    
    adata.write_h5ad(f'{root_path}{data_use}/data.h5ad')
    out_dataname_list.append(source_h5ad_data)
    out_h5ad_path_list.append(f'{data_use}/data.h5ad')
    out_feature_dict_list.append(f'{data_use}/feature_dict.pickle')
    N_Unit_list.append(adata.shape[0])
    N_Gene_list.append(adata.shape[1])
    
    print(source_h5ad_data)
    print(f'h5ad path: {data_use}/data.h5ad')
    print(f'把feature_dict写为: {data_use}/feature_dict.pickle')
    print(f'N_Unit: {adata.shape[0]}')
    print(f'N_Gene: {adata.shape[1]}')
    del adata
    gc.collect()

In [ ]:
np.array(out_dataname_list).dump('out_dataname_list.npy')
np.array(out_h5ad_path_list).dump('out_h5ad_path_list.npy')
np.array(out_feature_dict_list).dump('out_feature_dict_list.npy')
np.array(N_Unit_list).dump('N_Unit_list.npy')
np.array(N_Gene_list).dump('N_Gene_list.npy')

In [ ]:
1+1

In [ ]:
for i in out_dataname_list:
    print(i)

In [ ]:
for i in out_h5ad_path_list:
    print(i)

In [ ]:
for i in out_feature_dict_list:
    print(i)

In [ ]:
for i in N_Unit_list:
    print(i)

In [ ]:
for i in N_Gene_list:
    print(i)